### Lab 3.1: Basic Neural Network in PyTorch - Solution

Let's create a linear classifier one more time, but using PyTorch's automatic differentiation and optimization algorithms.  Then you will extend the perceptron into a multi-layer perceptron (MLP).

In [1]:
import numpy as np
import torch

We need to explicitly tell PyTorch when creating a tensor that we are interested in later computing its gradient

In [2]:
# Enable GPU acceleration
if torch.cuda.is_available():
    torch.set_default_device('cuda')

In [3]:
a = torch.tensor(5., requires_grad=True)
a

tensor(5., device='cuda:0', requires_grad=True)

In [4]:
b = torch.tensor(6., requires_grad=True)
c = 2*a+3*b
c

tensor(28., device='cuda:0', grad_fn=<AddBackward0>)

To extract the gradients, we first need to call `backward()`.

In [5]:
c.backward()

Now to get the gradient of any variable with respect to `c`, we simply access the `grad` attribute of that variable.

In [6]:
a.grad

tensor(2., device='cuda:0')

In [7]:
b.grad

tensor(3., device='cuda:0')

Let's load and format the Palmer penguins dataset for multi-class classification.

In [8]:
from palmerpenguins import load_penguins
from matplotlib import pyplot as plt

In [9]:
df = load_penguins()

# drop rows with missing values
df.dropna(inplace=True)

# get two features
X = df[['flipper_length_mm','bill_length_mm']].values

# convert species labels to integers
y = df['species'].map({'Adelie':0,'Chinstrap':1,'Gentoo':2}).values

To make the learning algorithm work more smoothly, we we will subtract the mean of each feature.

Here `np.mean` calculates a mean, and `axis=0` tells NumPy to calculate the mean over the rows (calculate the mean of each column).

In [10]:
X -= np.mean(X,axis=0)

Now we will convert our `X` and `y` arrays to torch Tensors.

In [11]:
X = torch.tensor(X).float()
y = torch.tensor(y).long()

In [12]:
from torch import nn

The `torch.nn.Sequential` class creates a feed-forward network from a list of `nn.Module` objects.  Here we provide a single `nn.Linear` class which performs an affine transformation ($Wx+b$) so that we will have a linear classifier.

In [13]:
linear_model = torch.nn.Sequential(
    torch.nn.Linear(2, 3), # two inputs, three outputs
)

Now we create a cross-entropy loss function object and a stochastic gradient descent (SGD) optimizer.

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()

In [15]:
lr = 1e-2
opt = torch.optim.SGD(linear_model.parameters(), lr=lr)

Finally we can iteratively optimize the model.

In [16]:
epochs = 100
for epoch in range(epochs):
    opt.zero_grad() # zero out the gradients

    z = linear_model(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    if epoch == 0 or epoch == epochs-1:
        print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 0.20828595757484436
epoch 99: loss is 0.13075871765613556


### Exercises

Extend the above code to implement an MLP with a single hidden layer of size 100.

Write code to compute the accuracy of each model.

Can you get the MLP to outperform the linear model?

In [17]:
mlp = torch.nn.Sequential(
    torch.nn.Linear(2, 100),
    torch.nn.Linear(100, 3),
)

lr = 1e-4
epochs = 100_000
mlp_opt = torch.optim.SGD(mlp.parameters(), lr=lr)

for epoch in range(epochs):
    mlp_opt.zero_grad() # zero out the gradients

    z = mlp(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    mlp_opt.step() # apply gradients
    
    if epoch == 0 or epoch == epochs-1:
        print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 3.1929023265838623
epoch 99999: loss is 0.11726874113082886


In [23]:
def accuracy(model, X, y):
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        z = torch.argmax(model(X), dim=1)
        return (z == y).float().mean().item()
        
    return None

In [24]:
print(f'Linear Model accuracy: {accuracy(linear_model, X, y)}')
print(f'Multi-Layer Perceptron accuracy: {accuracy(mlp, X, y)}')

Linear Model accuracy: 0.954954981803894
Multi-Layer Perceptron accuracy: 0.9579579830169678


Wohoo! MLP has a higher accuracy! (even if it's just a little)